# Tutorial 2 - Data Preprocessing

##### The following notebook is a modified version version of the talk given by April Chen in 2016 titled Depy 2016 Talk: Pre-Modeling: Data Preprocessing and Feature Exploration in Python. More information can be found from the original repo: https://github.com/aprilypchen/depy2016


### The objective of the tutorial is to show the importance of data manipulation, and how to apply common data pre-processing techniques to improve model performance.

In [1]:
# Bread and butter libraries to deal with dataframes and matrices
import numpy as np
import pandas as pd

##### For the following workshop, we will be using an edited version of the "adult" dataset from the public UCI repository. The dataset consists of information on various individuals, including age, education, marital status, gender, and income.

In [2]:
# Machine learning models cannot deal with null values, we will go over techniques to deal with them
na_values = ['#NAME?']
df = pd.read_csv('adult.csv', na_values=na_values)

In [3]:
# peek at first 10
df.head(10)

,age,workclass,fnlwgt,education,education_num,marital_status,occupation,relationship,race,sex,capital_gain,capital_loss,hours_per_week,native_country,income
0,39.0,State-gov,77516.0,Bachelors,13.0,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50.0,Self-emp-not-inc,83311.0,Bachelors,13.0,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38.0,Private,215646.0,HS-grad,9.0,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53.0,Private,234721.0,11th,7.0,Married-civ-spouse,Handlers-cleaners,Husband,Black,NaN,0,0,40,United-States,<=50K
4,28.0,Private,338409.0,Bachelors,13.0,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K
5,37.0,Private,284582.0,Masters,14.0,Married-civ-spouse,Exec-managerial,Wife,White,Female,0,0,40,United-States,<=50K
6,49.0,Private,160187.0,9th,5.0,Married-spouse-absent,Other-service,Not-in-family,Black,Female,0,0,16,Jamaica,<=50K
7,52.0,Self-emp-not-inc,209642.0,HS-grad,9.0,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,45,United-States,>50K
8,31.0,Private,45781.0,Masters,14.0,Never-married,Prof-specialty,Not-in-family,White,Female,14084,0,50,United-States,>50K
9,42.0,Private,159449.0,Bachelors,13.0,Married-civ-spouse,Exec-managerial,Husband,White,Male,5178,0,40,United-States,>50K


#### Binary Classification Problem: Predict, based on various features from the dataset, if someone's income is greater or less than 50k

In [4]:
# Observe dataset distribution, in practice when unbalanced datasets are not well treated, performance metrics can
# be very misleading
df['income'].value_counts()

<=50K    3779
>50K     1221
Name: income, dtype: int64

In [5]:
# Encode as 0 if income <=50K and as 1 if income >50K

df['income'] = [0 if x == '<=50K' else 1 for x in df['income']]

In [6]:
df.income.head(10)

0    0
1    0
2    0
3    0
4    0
5    0
6    0
7    1
8    1
9    1
Name: income, dtype: int64

## 1. Benchmark performance with unprocessed data

In [7]:
# Drop na values to not throw any errors
df_unprocessed = df
df_unprocessed = df_unprocessed.dropna(axis=0, how='any')

print(df.shape)
print(df_unprocessed.shape)

(5000, 15)
(4496, 15)


In [8]:
# Remove non-numeric columns so model doesn't throw errors
# Potential loss of information from categorical features is evident

for col_name in df_unprocessed.columns:
    if df_unprocessed[col_name].dtypes not in ['int32','int64','float32','float64']:
        df_unprocessed = df_unprocessed.drop(col_name, 1)

In [9]:
# Split into features and target variable
X_unprocessed = df_unprocessed.drop('income', 1)
y_unprocessed = df_unprocessed.income

In [10]:
X_unprocessed.head(5)

,age,fnlwgt,education_num,capital_gain,capital_loss,hours_per_week
0,39.0,77516.0,13.0,2174,0,40
1,50.0,83311.0,13.0,0,0,13
2,38.0,215646.0,9.0,0,0,40
4,28.0,338409.0,13.0,0,0,40
5,37.0,284582.0,14.0,0,0,40


### Import algo to measure baseline accuracy

In [11]:
# Import common ML tools from sklearn
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

### Split data into train/test split

In [12]:
X_train_unproc, X_test_unproc, y_train, y_test = train_test_split(
    X_unprocessed, y_unprocessed, train_size=0.70, test_size=0.30)

In [13]:
# Function that returns model accuracy
def find_model_perf(X_train, y_train, X_test, y_test):
    model = LogisticRegression()
    model.fit(X_train, y_train)
    pred = model.predict(X_test)
    acc = accuracy_score(y_test, pred)

    
    return acc

In [14]:
acc_unprocessed= find_model_perf(X_train_unproc, y_train, X_test_unproc, y_test)

In [15]:
acc_unprocessed

0.79614529280948854

#### NOTE: when feeding a test features into your prediction model, ensure test set has gone through the same preprocessing as your training set

## 2. Explore feature space to determine how to perform data pre-processing, then feed processed data into model to evaluate performance difference

In [16]:
# Separate features from target var.

y = df.income
X = df.drop(['income'], 1)

In [ ]:
y.head(15)

In [ ]:
X.head(5)

### Dealing with Categorical Data: One-hot encoding

#### One simple strategy to convert categorical data to numerical data is to create one dummy variable for each categorical value possible, then flagging with a 1 when the value is present

In [ ]:
pd.get_dummies(X['education']).head(5)

#### Determine how many possibles values there are for each feature

In [ ]:
# Decide which categorical variables you want to use in model 
for col_name in X.columns:
    if X[col_name].dtypes == 'object':
        unique_cat = len(X[col_name].unique())
        print("Feature '{col_name}' has {unique_cat} unique categories".format(col_name=col_name, unique_cat=unique_cat))

In [ ]:
# Investigate why there is a high number of unique values for 'native_country'
X['native_country'].value_counts().sort_values(ascending=False).head(10)

In [ ]:
# In this case, bucket low frequecy categories as "Other"
X['native_country'] = ['United-States ' if x == 'United-States' else 'Other' for x in X['native_country']]

print(X['native_country'].value_counts().sort_values(ascending=False))

#### Create list of important categorical features to encode

In [ ]:
# Create a list of features to dummy# Create 
todummy_list = ['workclass', 'education', 'marital_status', 'occupation', 'relationship', 'race', 'sex', 'native_country']

In [ ]:
# Function to dummy all the categorical variables used for modeling
def dummy_df(df, todummy_list):
    for x in todummy_list:
        dummies = pd.get_dummies(df[x], prefix=x, dummy_na=False)
        df = df.drop(x, 1)
        df = pd.concat([df, dummies], axis=1)
    return df

In [ ]:
X = dummy_df(X, todummy_list)
print(X.head(5))

### Investigate null values

In [ ]:
X.isnull().sum().sort_values(ascending=False).head()

In [ ]:
# Impute missing values using Imputer in sklearn.preprocessing
from sklearn.preprocessing import Imputer

imp = Imputer(missing_values='NaN', strategy='median', axis=0)
imp.fit(X)
X = pd.DataFrame(data=imp.transform(X) , columns=X.columns)

In [ ]:
# Sanity check
X.isnull().sum().sort_values(ascending=False).head()

### PCA to find most important components

In [ ]:
# Use PCA from sklearn.decompostion to find principal components
from sklearn.decomposition import PCA

pca = PCA(n_components=10)
X_pca = pd.DataFrame(pca.fit_transform(X))

In [ ]:
X_pca.head(10)

### Evaluate the same algorithm, but with preprocessed dataset

In [ ]:
from sklearn.cross_validation import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X_pca, y, train_size=0.70, random_state=1)

In [ ]:
acc_proc= find_model_perf(X_train, y_train, X_test, y_test)

In [ ]:
acc_proc

In [ ]:
improvement = np.round(((acc_proc - acc_unprocessed)/acc_unprocessed),3)*100

In [ ]:
improvement